<h1>Part B (Neural Network as Logic Gates)</h1>

In [1618]:
# Imports
import math
import torch
import numpy as np

In [1619]:
class NeuralNetwork():
    def __init__(self, layer_sizes):
        
        # Creating variables for the sizes of each layer
        self.layer_sizes = layer_sizes
        self.in_size = self.layer_sizes[0]
        self.out_size = self.layer_sizes[-1]
        self.hidden_sizes = self.layer_sizes[1:-1]
        self.num_layers = len(self.layer_sizes)
        
        # Build the weights for our network.
        # The layout for the weights will be a list of matrices for now.
        # The shape of a given weight item in the list will be [from layer size, to layer size]
        self.thetas = [0 for x in range(0,len(layer_sizes)-1)]
            
    def getLayer(self, theta, layer):
        return self.thetas[layer]
        
    def forward(self, input_list):
        
        z = []
        a = []
        
        first_layer_no_bias = torch.FloatTensor(input_list).view(1, len(input_list))
        
        z.append(first_layer_no_bias)
        a.append(z[0])
                        
        for i in range(0,self.num_layers-1):
            bias = torch.ones(1, 1)
            layer_with_bias = torch.cat([bias, a[i]], 1)
            z.append(torch.mm(layer_with_bias, self.thetas[i]))
            a.append(1/(1+np.exp(-z[-1])))

        result = a[-1][0]
        
        # If binary output desired:
        if result >= 0.5:
            result = 1.0
        elif result < 0.5:
            result = 0.0
        
        return result

<h2>Logic Gates</h2>

In [1620]:
class AND():
    def __init__(self):
        layer_sizes = [2, 1]
        self.network = NeuralNetwork(layer_sizes)

    def __call__(self, x, y):
        self.x = int(x)
        self.y = int(y)
        return self.forward()
    
    def getLayer(self, theta, layer):
        self.network.thetas[layer] = theta
        
    def forward(self):
        input_array = [self.x, self.y]
        result = self.network.forward(input_array)
        
        return bool(result)

In [1621]:
class OR():
    def __init__(self):       
        layer_sizes = [2, 1]
        self.network = NeuralNetwork(layer_sizes)

    def __call__(self, x, y):
        self.x = int(x)
        self.y = int(y)
        return self.forward()
    
    def getLayer(self, theta, layer):
        self.network.thetas[layer] = theta
        
    def forward(self):
        input_array = [self.x, self.y]
        result = self.network.forward(input_array)
        
        return bool(result)

In [1622]:
class NOT():
    def __init__(self):
        layer_sizes = [2, 1]
        self.network = NeuralNetwork(layer_sizes)

    def __call__(self, x):
        self.x = int(x)
        return self.forward()
    
    def getLayer(self, theta, layer):
        self.network.thetas[layer] = theta
        
    def forward(self):
        input_array = [self.x]
        result = self.network.forward(input_array)
        
        return bool(result)

In [1623]:
class XOR():
    def __init__(self):
        layer_sizes = [2, 2, 1]
        self.network = NeuralNetwork(layer_sizes)

    def __call__(self, x, y):
        self.x = int(x)
        self.y = int(y)
        return self.forward()
    
    def getLayer(self, theta, layer):
        self.network.thetas[layer] = theta
        
    def forward(self):
        input_array = [self.x, self.y]
        result = self.network.forward(input_array)
        
        return bool(result)

In [1624]:
And = AND()
Or = OR()
Not = NOT()
Xor = XOR()

# Custom Weights
# Rows = prev layer size
# Cols = next layer size
and_thetas = [torch.tensor([
    [-2.0], 
    [ 1.5], 
    [ 1.5]
]).float()]
or_thetas = [torch.tensor([
    [-0.25], 
    [ 1.5], 
    [ 1.5]
]).float()]
not_thetas = [torch.tensor([
    [ 0.0],
    [-1.0]
]).float()]
xor_thetas = [torch.tensor([
    [-0.25, 2.0],
    [ 1.5, -1.5],
    [ 1.5, -1.5]
]).float(), torch.tensor([
    [-2.0],
    [ 1.5],
    [ 1.5]
]).float()]

And.getLayer(and_thetas[0], 0)
Or.getLayer(or_thetas[0], 0)
Not.getLayer(not_thetas[0], 0)
Xor.getLayer(xor_thetas[0], 0)
Xor.getLayer(xor_thetas[1], 1)

print("AND result: ", And(False, False))
print("OR result: ", Or(False, True))
print("NOT result: ", Not(False)) 
print("XOR result: ", Xor(True, False))

AND result:  False
OR result:  True
NOT result:  True
XOR result:  True
